# Pulling SATCOM beams from UDL

Kerry N. Wood

kerry.wood@jhuapl.edu

May 5, 2022

Seems like data is contained in three main endpoints:

    - https://unifieddatalibrary.com/udl/channel
    - https://unifieddatalibrary.com/udl/beamcontour?idBeam=
    - https://unifieddatalibrary.com/udl/beam
    
Beam and channel we can **pull manually**.  Contour requires a parameter, and you have to loop on it.

In [ ]:
import requests
import base64
import time
import json

import pandas as pd
import geopandas as gpd

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Load pre-pulled data

In [ ]:
beams = pd.read_json('./data/beam_20220505.json')
beams.iloc[0]

In [ ]:
channels = pd.read_json('./data/channel_20220505.json')
channels.iloc[0]

In [ ]:
rfband = pd.read_json('./data/rfband_20220505.json')
rfband.iloc[0]

In [ ]:
# if you have a local copy, pull contours (these have to be pulled individually)
with open('./data/contours_20220505.json','r') as F: contours = json.load(F)

# Contour puller: only use if you haven't pulled already

- does ./data/contours... exist already?

## Consolidate and Map

- these data were split into these tables, but reference with `id_<whatever`... 
- klunky, but I think we can consolidate


- `contour` has the `idBeam` parameter that tells you the beam and a `gain` parameter as well
- `beam` maps `beamId` to the `beamName`
- `channel` has a `idTransponder` (that includes NORAD) that maps to `idRFBand`
- `idRFBand` will map to the frequency parameters

**TRY** start with channel...

In [ ]:
# get a working copy of the channels data
w_channels = channels[ ~ channels['idBeam'].isnull() ].copy()

# yank the NORAD data out of the transponder info
def getNorad( V ): 
    try: return int( V.split('-')[0] )
    except: return None

w_channels['NORAD'] = w_channels['idTransponder'].apply( getNorad )

#### With separate contours

In [ ]:
# merge the transponder (channels) with the beam data (beams / contours)
# NOTE: some channel / transponder info doesn't have beam data
w_final = w_channels.merge( beams, how='inner', left_on='idBeam', right_on='id', suffixes=('_channels','_beams') )

# now merge in the frequency data
w_final = w_channels.merge( rfband, how='inner', left_on='idRFBand', right_on='id', suffixes=('_channels','_rfband') )

w_final.to_json('./derived_data/udl_merged_no_contours.json')

In [ ]:
# merge code.. if you want to squish everything together
with open('./data/contours_20220505.json') as F: t_contours = json.load(F)
t_udl = pd.read_json('./derived_data/udl_merged_no_contours.json')
# get each contour
t_udl['contour'] = t_udl['idBeam'].apply( lambda X: contours.get(X,None) )

# Consolidated output : probably don't use

In [ ]:
# merge the transponder (channels) with the beam data (beams / contours)
# NOTE: some channel / transponder info doesn't have beam data
w_final = w_channels.merge( beams, how='inner', left_on='idBeam', right_on='id', suffixes=('_channels','_beams') )

# now merge in the frequency data
w_final = w_channels.merge( rfband, how='inner', left_on='idRFBand', right_on='id', suffixes=('_channels','_rfband') )

# get each contour
w_final['contour'] = w_final['idBeam'].apply( lambda X: contours.get(X,None) )

# w_final.to_json('udl_consolidated.json')